<a href="https://colab.research.google.com/github/chaosarium/3D-GANCA/blob/master/GANCA%20train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training GANCA-3D

In [ ]:
!wget -q https://github.com/chaosarium/3D-GANCA/raw/master/data_helper.py
!wget -q https://github.com/chaosarium/3D-GANCA/raw/master/utils.py
!wget -q https://github.com/chaosarium/3D-GANCA/raw/master/models.py
!wget -q https://github.com/chaosarium/3D-GANCA/raw/master/block_ids_alt.tsv
!wget -q https://github.com/chaosarium/3D-GANCA/raw/master/output.zip
!unzip -q output.zip
!wget -q https://github.com/chaosarium/3D-GANCA/blob/master/dataset/filtered_houses_stats.pkl4 -O dataset/filtered_houses_stats.pkl4
!wget -q https://github.com/chaosarium/3D-GANCA/blob/master/dataset/filtered_houses_stats.pkl -O dataset/filtered_houses_stats.pkl

dataset/filtered_houses_stats.pkl4: No such file or directory
dataset/filtered_houses_stats.pkl: No such file or directory


In [ ]:
|!pip --quiet install torchsummaryX loguru einops pytorch_lightning

     |████████████████████████████████| 58 kB 4.3 MB/s 
     |████████████████████████████████| 527 kB 17.2 MB/s 
     |████████████████████████████████| 134 kB 88.6 MB/s 
     |████████████████████████████████| 596 kB 69.7 MB/s 
     |████████████████████████████████| 397 kB 87.2 MB/s 
     |████████████████████████████████| 829 kB 86.4 MB/s 
     |████████████████████████████████| 952 kB 77.2 MB/s 
     |████████████████████████████████| 1.1 MB 51.0 MB/s 
     |████████████████████████████████| 271 kB 73.5 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 144 kB 79.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
# import stuff

import data_helper
import utils

%load_ext autoreload
%autoreload 2

import numpy as np
import sys
import pickle
import torch
import random
from torchsummary import summary
import matplotlib.pyplot as plt

from loguru import logger as gurulogger
gurulogger.remove()
gurulogger.add(sys.stdout, colorize=True, format="<blue>{time}</blue> <level>{message}</level>")
gurulogger.level("INFO", color="<red><bold>")


import os
from einops import rearrange
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

BLOCK2VEC_OUT_PATH = 'output/block2vec saves/block2vec 64 dim/'
NUM_WORKERS = int(os.cpu_count() / 2)

import pytorch_lightning as pl

if torch.cuda.is_available():
    print('CUDNN VERSION:', torch.backends.cudnn.version())
    print('Number CUDA Devices:', torch.cuda.device_count())
    print('CUDA Device Name:',torch.cuda.get_device_name(0))
    print('CUDA Device Total Memory [GB]:',torch.cuda.get_device_properties(0).total_memory/1e9)

CUDNN VERSION: 8005
Number CUDA Devices: 1
CUDA Device Name: Tesla P100-PCIE-16GB
CUDA Device Total Memory [GB]: 17.071734784


In [ ]:
embedding, mcid2block, block2embeddingidx, embeddingidx2block = utils.get_embedding_info(BLOCK2VEC_OUT_PATH)

In [ ]:
from data_helper import GANCA3DDataModule

In [ ]:
from models import VoxelPerceptionNet, VoxelUpdateNet, VoxelNCAModel, VoxelDiscriminator
from pytorch_lightning import Trainer

In [ ]:
%load_ext tensorboard
# %tensorboard --logdir lightning_logs/

In [ ]:
class GANCA(pl.LightningModule):

    def __init__(self,
            lr = 2e-4,
            beta1 = 0.9,
            beta2 = 0.999,
            batch_size = 16,
            num_embedding_channels = 64,
            num_hidden_channels = 63,
            update_net_channel_dims = [32, 32],
            embedding: torch.nn.Embedding = None,
            step_range = [16, 20],
        ):
        
        super().__init__()
        # call this to save args to the checkpoint
        self.save_hyperparameters()
        
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.batch_size = batch_size
        self.num_embedding_channels = num_embedding_channels
        self.num_hidden_channels = num_hidden_channels
        self.update_net_channel_dims = update_net_channel_dims
        # the channels will be like [alpha, embeddings ... , hiddens ...]
        self.num_channels = 1 + self.num_embedding_channels + self.num_hidden_channels
        self.world_size = (32,32,32)
        self.embedding = embedding
        self.embedding.weight.requires_grad=False # freeze embeddings
        self.step_range = step_range
        
        self.generator = VoxelNCAModel(
            alpha_living_threshold = 0.1,
            cell_fire_rate = 0.5,
            num_perceptions = 3,
            perception_requires_grad = True,
            num_embedding_channels = self.num_embedding_channels,
            num_hidden_channels = self.num_hidden_channels,
            normal_std = 0.0002,
            use_normal_init = True,
            zero_bias = True,
            update_net_channel_dims = self.update_net_channel_dims,
        )
        self.discriminator = VoxelDiscriminator(
            num_in_channels = self.num_embedding_channels, 
            use_sigmoid=True,
        )
        
        # generate some random seeds (N, channels, x, y, z)
        self.validation_noise = self.make_seed_states(16)
        
    def make_seed_states(self, batch_size):
        return utils.make_seed_state(
            batch_size = batch_size,
            num_channels = self.num_channels, 
            alpha_channel_index = 0,
            seed_dim = (4, 4, 4), 
            world_size = self.world_size,
        )
    
    def training_step(self, batch, batch_idx, optimizer_idx):
        
        num_steps = random.randint(*self.step_range)
            
        real_houses = batch
        type_holder = batch[0,0,0,0].to(torch.float) # this is a dummy type for creating labels
        # gurulogger.info(f'created typeholder tensor {type_holder} for new tensors with type {type_holder.type()} on device {type_holder.get_device()}')
        size_this_batch = real_houses.shape[0]
        # gurulogger.info(f'got shape {real_houses.shape} as real example')
                
        # make noise
        
        seed_states = self.make_seed_states(size_this_batch).type_as(type_holder) # same batch size as those coming in
        # gurulogger.info(f'made seed states with shape {seed_states.shape} and type {seed_states.type()}')
            
        # train generator
        if optimizer_idx == 0:
            
            # gurulogger.info("ENTERING GENERATOR TRAIN LOOP")

            # generate images
            fake_houses_states = self.generator.forward(seed_states, steps=num_steps)
            # gurulogger.info(f'generated houses of shape {fake_houses_states.shape}')
                        
            # create ground truth result (all fake results) we want D to say the generated ones are real so they are all 1s
            real_labels = utils.make_real_labels(size_this_batch).type_as(type_holder)
            # gurulogger.info(f'real labels looks like: {real_labels}')
            
            # now get the embedding parts out of the fake states
            fake_houses = fake_houses_states[:, 1:self.num_embedding_channels+1, :, :, :]
            # gurulogger.info(f'extracted fake_houses with shape: {fake_houses.shape}')
            
            # see what discriminator thinks
            fake_predictions = self.discriminator.forward(fake_houses)
            # gurulogger.info(f"the discriminator predicted {fake_predictions} for the houses. let's compare that to the real label {real_labels}")
            
            # calculate loss
            g_loss = F.binary_cross_entropy(fake_predictions, real_labels) # y_hat, y  
            # gurulogger.info(f'gooooooooooooooooooooooooooooooooooooooood so far for generator. step done with g_loss **{g_loss}**')
                                    
            self.log("g_loss", g_loss.detach(), prog_bar=True, logger=True)

            return g_loss
        
        if optimizer_idx == 1:
            
            # gurulogger.info("ENTERING DISCRIMINATOR TRAIN LOOP")
            
            # get embeddings for the real house
            real_houses = utils.examples2embedding(real_houses, self.embedding)
            
            # gurulogger.info(f"got embedded real houses with shape {real_houses.shape}")

            # pass in real image to D and try to make it predict all 1s
            real_targets = utils.make_real_labels(size_this_batch).type_as(type_holder)
            real_predictions = self.discriminator.forward(real_houses)
            # gurulogger.info(f"on a real house, D gave predictions: {real_predictions}; and we want to match the label {real_targets}")
            real_loss = F.binary_cross_entropy(real_predictions, real_targets)
            # TODO fix accuracy formula
            real_acc = torch.mean(real_predictions).item() # the average prediction. The closer to 1 the more accurate
            # gurulogger.info(f"real loss is therefore: {real_loss}; and real accuracy is {real_acc}")
            
            # meanwhile, we also want D to be able to tell that outputs from G are all fake
            fake_targets = utils.make_fake_labels(size_this_batch).type_as(type_holder)
            fake_houses_states = self.generator.forward(seed_states, steps=num_steps).detach() # detach so that gradients don't pass back into generator
            fake_houses = fake_houses_states[:, 1:self.num_embedding_channels+1, :, :, :] # extract the embedding parts
            # gurulogger.info(f"created fake target: {fake_targets}; generated from seeds fake houses output {fake_houses_states.shape} and extracted the embeddings to {fake_houses.shape}")
            fake_predictions = self.discriminator.forward(fake_houses)
            # gurulogger.info(f"D predicted: {fake_predictions} on fake houses")
            fake_loss = F.binary_cross_entropy(fake_predictions, fake_targets)
            # TODO fix accuracy formula
            fake_acc = 1 - torch.mean(fake_predictions).item() # the 1 - average prediction. The closer to 1 the more accurate
            # gurulogger.info(f"fake loss is therefore: {fake_loss}; and fake accuracy is {fake_acc}")
                        
            # discriminator loss is the average of the two losses
            d_loss = (real_loss + fake_loss) / 2
            avg_acc = (real_acc + fake_acc) / 2
            # gurulogger.info(f"avg D loss is: {d_loss}; and avg accuracy is {avg_acc}")
                    
            self.log("fake_loss", fake_loss.detach(), prog_bar=False, logger=True)
            self.log("real_loss", real_loss.detach(), prog_bar=False, logger=True)
            self.log("d_loss", d_loss.detach(), prog_bar=True, logger=True)
            self.log("real_acc", real_acc, prog_bar=False, logger=True)
            self.log("fake_acc", fake_acc, prog_bar=False, logger=True)
            self.log("avg_acc", avg_acc, prog_bar=True, logger=True)

            return d_loss

    def configure_optimizers(self,):
        # define pytorch optimizers here. return [list of optimizers], [list of LR schedulers]
        g_optimizer = torch.optim.Adam(self.generator.parameters(), lr=self.lr, betas=(self.beta1, self.beta2))
        d_optimizer = torch.optim.Adam(self.discriminator.parameters(), lr=self.lr, betas=(self.beta1, self.beta2))
        return [g_optimizer, d_optimizer], []

In [ ]:
model = GANCA(
    lr = 2e-4,
    beta1 = 0.9,
    beta2 = 0.999,
    batch_size = 8,
    num_embedding_channels = 64,
    num_hidden_channels = 63,
    update_net_channel_dims = [16, 16],
    embedding = embedding,
    step_range = [16, 20],
)

In [ ]:
dm = GANCA3DDataModule(batch_size=8, num_workers=1, mcid2block = mcid2block, block2embeddingid = block2embeddingidx, debug=False)
dm.prepare_data()
dm.setup()

Dataset already exists


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:175: LightningDeprecationWarning: DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.
  rank_zero_deprecation("DataModule property `dims` was deprecated in v1.5 and will be removed in v1.7.")


  0%|          | 0/1977 [00:00<?, ?it/s]

loaded 1977 houses
Turning MC id into embedding idx. This could take up to a minute.


In [ ]:
logger = pl.loggers.TensorBoardLogger(save_dir='lightning_logs', name='GANCA', default_hp_metric=False)
trainer = Trainer(gpus=1, max_epochs=1, fast_dev_run=False, log_every_n_steps=1, logger=logger, profiler="simple")

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [ ]:
trainer.fit(model, dm)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/configuration_validator.py:120: UserWarning: You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.
  rank_zero_warn("You passed in a `val_dataloader` but have no `validation_step`. Skipping val loop.")
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/core/datamodule.py:470: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  f"DataModule.{name} has already been called, so it will not be called again. "
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Missing logger folder: lightning_logs/GANCA

  | Name          | Type               | Params
-----------------------------------------------------
0 | embedding     | Embedding          | 14.0 K
1 | generator     | VoxelNCAModel      | 18.8 K
2 | discriminator | VoxelDiscriminator | 11.3 M
---------------------------------------

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:688: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


---
debug

---

In [ ]:
data_helper.download_dataset()

Dataset already exists
Extracting dataset to dataset/house_data


In [ ]:
sample_state = torch.rand(1, 128, 32, 32, 32)
sample_seed_state = utils.make_seed_state(
    batch_size = 1,
    num_channels = 128, 
    alpha_channel_index = 0,
    seed_dim = (4, 4, 4), 
    world_size = (32,32,32),
)

In [ ]:
# before train
out1 = model.generator.forward(sample_state, steps=1)
torch.sum(out1)

2022-02-28T20:32:09.898136+0800 it's shape torch.Size([1, 128, 32, 32, 32]) coming in to the perception net
2022-02-28T20:32:10.059550+0800 it's shape torch.Size([1, 384, 32, 32, 32]) coming in to the update net


tensor(2097196.2500, grad_fn=<SumBackward0>)

In [ ]:
# after train
out3 = model.generator.forward(sample_state, steps=1)
torch.sum(out3)

2022-02-28T20:32:54.688500+0800 it's shape torch.Size([1, 128, 32, 32, 32]) coming in to the perception net
2022-02-28T20:32:54.819261+0800 it's shape torch.Size([1, 384, 32, 32, 32]) coming in to the update net


tensor(2097196.2500, grad_fn=<SumBackward0>)

In [ ]:
torch.equal(out1, out3)

False

In [ ]:
out2 = model.generator.forward(sample_state, steps=8)
torch.sum(out2)

tensor(2097251.2500, grad_fn=<SumBackward0>)